In [1]:
import zipfile
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import optuna
from plotly.io import show
from sklearn.model_selection import train_test_split

from sklearn.model_selection import TimeSeriesSplit

from skfolio import PerfMeasure, RatioMeasure, RiskMeasure
from skfolio.datasets import load_sp500_dataset
from skfolio.optimization import MeanRisk,ObjectiveFunction
from skfolio.preprocessing import prices_to_returns
from skfolio.model_selection import WalkForward, cross_val_predict
from skfolio import MultiPeriodPortfolio, Population, Portfolio
from skfolio.optimization import EqualWeighted, MaximumDiversification,InverseVolatility 
from skfolio.measures import PerfMeasure






c:\Users\bodya\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning:

TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.

c:\Users\bodya\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning:

Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.

c:\Users\bodya\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning:

TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.



In [ ]:

def unzip_file(zip_path, extract_to='.'):
    """
    Разархивирует ZIP-архив в указанную директорию.
    
    :param zip_path: Путь к ZIP-архиву
    :param extract_to: Директория для извлечения (по умолчанию текущая)
    """
    try:
        # Создать директорию, если она не существует
        os.makedirs(extract_to, exist_ok=True)
        
        # Открыть ZIP-архив
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # Извлечь все файлы
            zip_ref.extractall(extract_to)
            print(f'Архив успешно извлечен в: {extract_to}')
            
    except FileNotFoundError:
        print(f'Ошибка: файл {zip_path} не найден.')
    except zipfile.BadZipFile:
        print('Ошибка: файл не является ZIP-архивом или архив поврежден.')
    except Exception as e:
        print(f'Произошла ошибка: {str(e)}')

# Пример использования
unzip_file('D1.zip', extract_to='D1_data')
unzip_file('H1.zip', extract_to='H1_data')


In [2]:


def process_folder_data(folder_path,date_col='date', start_date='2010-01-01', threshold=0.01):
    """
    Обрабатывает все CSV-файлы в указанной папке, объединяет данные,
    фильтрует по дате и удаляет столбцы с пропусками.

    Параметры:
    - folder_path: путь к папке с CSV-файлами
    - start_date: начальная дата для фильтрации (по умолчанию '2010-01-01')
    - threshold: максимально допустимая доля пропусков (по умолчанию 0.01)

    Возвращает:
    - Объединенный и очищенный DataFrame
    """
    
    data_frames = []

    # Обработка каждого CSV-файла
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)
            
            # Загрузка и преобразование данных
            df = pd.read_csv(file_path)
            df['datetime'] = pd.to_datetime(df[date_col])
            df['tic'] = file_name.replace('.csv', '')
            
            # Преобразование в широкий формат
            df_pivot = df.pivot(index='datetime', columns='tic', values='close')
            data_frames.append(df_pivot)

    # Объединение данных
    merged_data = pd.concat(data_frames, axis=1)
    
    # Фильтрация по дате
    merged_data = merged_data.loc[start_date:]
    
    # Удаление столбцов с пропусками
    null_percent = merged_data.isna().mean()
    columns_to_drop = null_percent[null_percent >= threshold].index
    merged_data = merged_data.drop(columns=columns_to_drop)

    return merged_data

# Вызов функции
result = process_folder_data(
    folder_path="data_daily",
    start_date='2010-01-01',
    threshold=0.03
)

# Просмотр первых 5 строк
print(result.head())

tic           ABIO    AFKS   AFLT    AKRN   AMEZ   APTK    BLNG   BSPB   CHMF  \
datetime                                                                        
2011-11-17  11.600  23.500  51.00  1410.0  9.280  48.99  11.650  93.88  428.2   
2011-11-18  11.605  22.923  50.74  1374.1  9.006  48.98  11.456  89.00  426.1   
2011-11-21  11.167  21.750  50.00  1395.0  8.809  47.49  10.980  82.99  395.5   
2011-11-22  10.965  21.902  49.90  1438.0  8.806  47.00  10.850  81.25  404.1   
2011-11-23  10.770  23.750  50.00  1480.3  8.978  46.88  10.900  81.00  406.0   

tic           CHMK  ...   SNGSP   SVAV    TATN  TATNP      TGKA      TGKB  \
datetime            ...                                                     
2011-11-17  3521.0  ...  16.299  354.3  157.05  97.21  0.010397  0.003609   
2011-11-18  3492.0  ...  16.330  356.0  154.14  95.01  0.010272  0.003560   
2011-11-21  3336.0  ...  15.770  334.2  143.70  93.00  0.009991  0.003410   
2011-11-22  3344.0  ...  15.869  334.9  148.26 

tic,ABIO,AFKS,AFLT,AKRN,AMEZ,APTK,BLNG,BSPB,CHMF,CHMK,...,SNGSP,SVAV,TATN,TATNP,TGKA,TGKB,TRMK,TRNFP,VSMO,VTBR
datetime,,,,,,,,,,,,,,,,,,,,,
2011-11-17,11.600,23.500,51.00,1410.0,9.280,48.990,11.650,93.88,428.2,3521.0,...,16.299,354.3,157.05,97.21,0.010397,0.003609,90.50,474.70,5010.0,345.40
2011-11-18,11.605,22.923,50.74,1374.1,9.006,48.980,11.456,89.00,426.1,3492.0,...,16.330,356.0,154.14,95.01,0.010272,0.003560,88.18,463.58,4903.8,340.00
2011-11-21,11.167,21.750,50.00,1395.0,8.809,47.490,10.980,82.99,395.5,3336.0,...,15.770,334.2,143.70,93.00,0.009991,0.003410,82.68,460.55,4699.5,316.15
2011-11-22,10.965,21.902,49.90,1438.0,8.806,47.000,10.850,81.25,404.1,3344.0,...,15.869,334.9,148.26,96.71,0.010087,0.003437,83.70,462.80,4747.1,318.50
2011-11-23,10.770,23.750,50.00,1480.3,8.978,46.880,10.900,81.00,406.0,3401.0,...,15.950,329.0,146.67,96.00,0.010079,0.003480,81.51,464.04,4710.0,313.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-14,83.720,14.919,68.69,16250.0,NaN,10.310,14.370,383.46,1035.0,5100.0,...,51.700,666.5,686.40,648.60,0.006698,0.006650,110.44,1236.80,31120.0,75.50
2025-04-15,83.740,14.736,69.35,16236.0,NaN,10.248,14.430,384.20,1042.0,5125.0,...,52.030,664.0,684.10,646.20,0.006564,0.006620,110.14,1267.20,31400.0,77.10
2025-04-16,83.520,14.927,69.35,16400.0,NaN,10.358,14.760,388.30,1041.4,5310.0,...,52.305,667.5,685.90,648.00,0.006598,0.006680,110.10,1286.60,31800.0,79.73


In [3]:

X = prices_to_returns(result.fillna(method='ffill'))
X_train=X.loc[:'2022-07-01']
X_test=X.loc['2022-07-01':]


#X_train, X_test = train_test_split(X, test_size=0.15, shuffle=False)
base_costs = {col: 0.003 for col in X.columns}


transaction_costs = {company: base_costs.get(company) / 21 for company in X.columns}

C:\Users\bodya\AppData\Local\Temp\ipykernel_17072\934785917.py:1: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



# Равновесный портфель 

In [21]:
bench = EqualWeighted()
bench.fit(X_test)
pred=bench.predict(X_test)


population = Population([pred])
print(f"Mean return: {pred.mean}")
print(f"Max drawdown: {pred.max_drawdown}")
print(f"Average drawdown: {pred.average_drawdown}")
print(f"Sharpe:  {pred.sharpe_ratio}")
print(f"Sortino:  {pred.sortino_ratio}")
print(f"Final return: {pred.cumulative_returns[-1]}")
print(f"Max return: {pred.cumulative_returns.max()}")
fig = population.plot_cumulative_returns()
show(fig)




Mean return: 0.0007200830380482622
Max drawdown: 0.46285552135953345
Average drawdown: 0.11247367178541673
Sharpe:  0.050852067424347516
Sortino:  0.0677722439018675
Final return: 0.5220602025849903
Max return: 0.7748117487318625


In [11]:
pred.plot_composition()

# Марковицевский подход

In [20]:

model = MeanRisk(
    # Минимизируем волатильность (можно менять на другие цели)
    objective_function=ObjectiveFunction.MINIMIZE_RISK,
    
    transaction_costs=transaction_costs,
    # Ограничения на веса: 0% <= вес актива <= 100%
    min_weights=0.0, 
    max_weights=1.0,
    # Ограничение на сумму весов = 100%
    budget=1.0,
)

# Обучение модели на исторических данных
model.fit(X_train)
pred=model.predict(X_test)


population = Population([pred])

print(f"Mean return: {pred.mean}")
print(f"Max drawdown: {pred.max_drawdown}")
print(f"Average drawdown: {pred.average_drawdown}")
print(f"Sharpe:  {pred.sharpe_ratio}")
print(f"Sortino:  {pred.sortino_ratio}")
print(f"Final return: {pred.cumulative_returns[-1]}")
print(f"Max return: {pred.cumulative_returns.max()}")
fig = population.plot_cumulative_returns()
show(fig)


Mean return: 0.00039656064921932615
Max drawdown: 0.455556137332147
Average drawdown: 0.11157573920127521
Sharpe:  0.029707333150870138
Sortino:  0.04033602240899067
Final return: 0.28750647068401136
Max return: 0.5735749880659465


In [9]:
pred.plot_composition()

## ищем оптимум на cv 

In [44]:
holding_period = 60
fitting_period = 128
cv = WalkForward(train_size=fitting_period, test_size=holding_period)

pred3 = MultiPeriodPortfolio(name="pred3",fitness_measures=[PerfMeasure.MEAN,RiskMeasure.MAX_DRAWDOWN])

model = MeanRisk(transaction_costs=transaction_costs, l1_coef=0.01,min_weights = 0 )
model.set_params(transaction_costs=transaction_costs)
previous_weights = None
for train, test in cv.split(X_train):
    X_train_cv = X_train.take(train)
    X_test_cv  = X_train.take(test)
    model.set_params(previous_weights=previous_weights)
    model.fit(X_train_cv)
    portfolio = model.predict(X_test_cv)
    pred3.append(portfolio)
    previous_weights = model.weights_

In [38]:
test_return=model.set_params(transaction_costs=transaction_costs,previous_weights=previous_weights).predict(X_test)
population_test = Population([test_return])
fig = population_test.plot_cumulative_returns()



print(f"Mean return: {test_return.mean}")
print(f"Max drawdown: {test_return.max_drawdown}")
print(f"Average drawdown: {test_return.average_drawdown}")
print(f"Sharpe:  {test_return.sharpe_ratio}")
print(f"Sortino:  {test_return.sortino_ratio}")
print(f"Final return: {test_return.cumulative_returns[-1]}")
print(f"Max return: {test_return.cumulative_returns.max()}")

show(fig)

Mean return: 0.0014103419728327855
Max drawdown: 0.4297702698172068
Average drawdown: 0.14532187696145732
Sharpe:  0.06387421454782154
Sortino:  0.10856368870218433
Final return: 1.02249793030377
Max return: 1.3264246702878653


In [39]:
population_test.plot_composition()

In [ ]:
[round(a,4) for a in model.weights_]

1.0

# Динамическое обучение модели с переопределением весов потрфеля

In [61]:
def run_dynamic_strategy(X_test, X_train, transaction_costs, model_class, model_params,
                         holding_period=60, fitting_period=128, portfolio_name="test_dynamic_portfolio",prev_weights=[round(a,5) for a in model.weights_]):
    """
    Запускает динамическую стратегию портфеля с заданными параметрами
    
    Параметры:
    X_test - тестовые данные
    X_train - тренировочные данные
    transaction_costs - транзакционные издержки
    model_class - класс модели оптимизации
    model_params - параметры для инициализации модели
    holding_period - период удержания позиций
    fitting_period - период обучения модели
    portfolio_name - название портфеля
    """
    # Инициализация портфеля
    portfolio = MultiPeriodPortfolio(
        name=portfolio_name,
        fitness_measures=[PerfMeasure.MEAN, RiskMeasure.CVAR]
    )

    if prev_weights is None:
    # Начальные веса
        prev_weights = np.ones(len(X_test.columns)) / len(X_test.columns)
    

    # Создаем генератор окон
    wf = WalkForward(train_size=fitting_period, test_size=holding_period)

    # Основной цикл обработки
    for i, (train_idx, test_idx) in enumerate(wf.split(X_test)):
        # Формирование обучающих данных
        if i == 0 and len(train_idx) < fitting_period:
            train_data = pd.concat([
                X_train.iloc[-fitting_period + len(train_idx):],
                X_test.take(train_idx)
            ])
        else:
            train_data = X_test.take(train_idx)

        test_data = X_test.take(test_idx)

        # Инициализация модели с учетом предыдущих весов
        model = model_class(
            **model_params,
            previous_weights=prev_weights
        )

        # Обучение и прогнозирование
        model.fit(train_data)
        result = model.predict(test_data)

        # Обновление портфеля и весов
        portfolio.append(result)
        prev_weights = model.weights_

    # Визуализация результатов
    pop = Population([portfolio])
    
    print(f"\nPerformance metrics for {portfolio_name}:")
    print(f"Mean return: {portfolio.mean}")
    print(f"Max drawdown: {portfolio.max_drawdown}")
    print(f"Average drawdown: {portfolio.average_drawdown}")
    print(f"Sharpe: {portfolio.sharpe_ratio}")
    print(f"Sortino: {portfolio.sortino_ratio}")
    print(f"Final return: {portfolio.cumulative_returns[-1]}")
    print(f"Max return: {portfolio.cumulative_returns.max()}")

    show(pop.plot_cumulative_returns())
    show(pop.plot_composition())

    return portfolio, pop

In [71]:
# Общие параметры
#Best parameters: {'l1_coef': 1.132971953693404e-05, 'l2_coef': 1.4258748847106347e-05, 'holding_period': 6, 'fitting_period': 95}
common_params = {
    'transaction_costs': transaction_costs,
    #'l1_coef': 0.0023162754506576176,
    'l2_coef': 0.09764064961117494
}

# Параметры для MeanRisk
mean_risk_params = {
    **common_params,
    'objective_function': ObjectiveFunction.MAXIMIZE_RETURN
}

# Запуск MeanRisk стратегии
mean_risk_portfolio, _ = run_dynamic_strategy(
    X_test=X_test,
    X_train=X_train,
    transaction_costs=transaction_costs,
    model_class=MeanRisk,
    model_params=mean_risk_params,
    holding_period=3, fitting_period=64,
    portfolio_name="MeanRisk Portfolio"
)

# Запуск MaximumDiversification стратегии
max_div_portfolio, _ = run_dynamic_strategy(
    X_test=X_test,
    X_train=X_train,
    transaction_costs=transaction_costs,
    model_class=MaximumDiversification,
    model_params=common_params,
    holding_period=3, fitting_period=64,
    portfolio_name="MaxDiversification Portfolio"
)


Performance metrics for MeanRisk Portfolio:
Mean return: 0.000848352440335491
Max drawdown: 0.5493036548797188
Average drawdown: 0.17973224897409332
Sharpe: 0.05598117863858895
Sortino: 0.07747136551612809
Final return: 0.559912610621423
Max return: 0.9846593166325333



Performance metrics for MaxDiversification Portfolio:
Mean return: 0.0005450870201810609
Max drawdown: 0.5505514916575631
Average drawdown: 0.1760236239178522
Sharpe: 0.04052862195557047
Sortino: 0.05845601332834022
Final return: 0.3597574333194997
Max return: 0.7914599332330811


In [74]:
max_div_portfolio.plot_contribution(PerfMeasure.MEAN)

# Подбор гипер параметров 

In [66]:

#from portfolio_optimization import MultiPeriodPortfolio, MaximumDiversification, PerfMeasure, RiskMeasure, Population, WalkForward

def objective(trial):
    # Define the hyperparameters to optimize
    l1_coef = trial.suggest_float('l1_coef', 1e-5, 1e-1, log=True)
    l2_coef = trial.suggest_float('l2_coef', 1e-5, 1e-1, log=True)
    
    # Also optimize the holding and fitting periods
    holding_period = trial.suggest_int('holding_period', 1, 30)
    fitting_period = trial.suggest_int('fitting_period', 30, 240)
    
    # Dynamic portfolio evaluation with current parameters
    test_dynamic_portfolio = MultiPeriodPortfolio(
        name=f"test_portfolio_{trial.number}",
        fitness_measures=[PerfMeasure.MEAN, RiskMeasure.CVAR]
    )
    
    # Initial weights - equal distribution
    previous_weights = np.ones(len(X.columns)) / len(X.columns)
    
    # Create a CV split for test data with the current periods
    test_cv = WalkForward(train_size=fitting_period, test_size=holding_period)
    
    # Iterate through all test periods
    for i, (train_idx, test_idx) in enumerate(test_cv.split(X_test)):
        # For the first period, we might need to use part of the training data
        if i == 0 and len(train_idx) < fitting_period:
            # Take the last data from the training set
            train_data = pd.concat([
                X_train.iloc[-fitting_period + len(train_idx):],
                X_test.iloc[train_idx]
            ])
        else:
            train_data = X_test.iloc[train_idx]
        
        test_data = X_test.iloc[test_idx]
        
        # Initialize the model with current trial parameters
        model = MaximumDiversification(
            transaction_costs=transaction_costs, 
            previous_weights=previous_weights,
            l1_coef=l1_coef,
            l2_coef=l2_coef,
        )
        
        # Train the model on the current training window
        model.fit(train_data)
        
        # Predict weights for the current test window
        portfolio = model.predict(test_data)
        
        # Add results to the multi-period portfolio
        test_dynamic_portfolio.append(portfolio)
        
        # Update weights for the next iteration
        previous_weights = model.weights_
    
    # Return the average drawdown to minimize
    return test_dynamic_portfolio.average_drawdown

# Main optimization function
def optimize_portfolio_parameters(n_trials=20):
    # Create a study object and optimize the objective function
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)
    
    # Get the best parameters
    best_params = study.best_params
    print(f"Best parameters: {best_params}")
    print(f"Best average drawdown: {study.best_value}")
    
    # Extract the best parameters
    best_l1_coef = best_params['l1_coef']
    best_l2_coef = best_params['l2_coef']
    best_holding_period = best_params['holding_period']
    best_fitting_period = best_params['fitting_period']
    
    # Visualization of optimization results
    fig1 = optuna.visualization.plot_optimization_history(study)
    fig2 = optuna.visualization.plot_param_importances(study)
    
    # Plot parallel coordinate plot to visualize parameter relationships
    fig3 = optuna.visualization.plot_parallel_coordinate(study)
    
    # Create the final portfolio with the best parameters
    final_portfolio = evaluate_portfolio(
        best_l1_coef, 
        best_l2_coef, 
        best_holding_period, 
        best_fitting_period
    )
    
    return study, final_portfolio

# Function to evaluate a portfolio with specific parameters
def evaluate_portfolio(l1_coef, l2_coef, holding_period, fitting_period):
    test_dynamic_portfolio = MultiPeriodPortfolio(
        name="best_portfolio",
        fitness_measures=[PerfMeasure.MEAN, RiskMeasure.CVAR]
    )
    
    previous_weights = np.ones(len(X.columns)) / len(X.columns)
    test_cv = WalkForward(train_size=fitting_period, test_size=holding_period)
    
    for i, (train_idx, test_idx) in enumerate(test_cv.split(X_test)):
        if i == 0 and len(train_idx) < fitting_period:
            train_data = pd.concat([
                X_train.iloc[-fitting_period + len(train_idx):],
                X_test.iloc[train_idx]
            ])
        else:
            train_data = X_test.iloc[train_idx]
        
        test_data = X_test.iloc[test_idx]
        
        model = MaximumDiversification(
            transaction_costs=transaction_costs, 
            previous_weights=previous_weights,
            l1_coef=l1_coef,
            l2_coef=l2_coef,
        )
        
        model.fit(train_data)
        portfolio = model.predict(test_data)
        test_dynamic_portfolio.append(portfolio)
        previous_weights = model.weights_
    
   
    population_test = Population([test_dynamic_portfolio])
    fig = population_test.plot_cumulative_returns()
    plt.show()
    
    
    print(f"Mean return: {test_dynamic_portfolio.mean}")
    print(f"Max drawdown: {test_dynamic_portfolio.max_drawdown}")
    print(f"Average drawdown: {test_dynamic_portfolio.average_drawdown}")
    print(f"Sharpe ratio: {test_dynamic_portfolio.sharpe_ratio}")
    print(f"Sortino ratio: {test_dynamic_portfolio.sortino_ratio}")
    print(f"Calmar ratio: {test_dynamic_portfolio.calmar_ratio}")
    
    return test_dynamic_portfolio


if __name__ == "__main__":
    # Constants
    transaction_costs = 0.001 
    
    
    study, best_portfolio = optimize_portfolio_parameters(n_trials=30)
    
    # Save the best parameters
    with open("best_portfolio_params.txt", "w") as f:
        f.write(f"Best parameters:\n")
        f.write(f"  l1_coef: {study.best_params['l1_coef']}\n")
        f.write(f"  l2_coef: {study.best_params['l2_coef']}\n")
        f.write(f"  holding_period: {study.best_params['holding_period']}\n")
        f.write(f"  fitting_period: {study.best_params['fitting_period']}\n\n")
        f.write(f"Best average drawdown: {study.best_value}\n\n")
        f.write(f"Performance metrics:\n")
        f.write(f"  Mean return: {best_portfolio.mean}\n")
        f.write(f"  Max drawdown: {best_portfolio.max_drawdown}\n")
        f.write(f"  Average drawdown: {best_portfolio.average_drawdown}\n")
        f.write(f"  Sharpe ratio: {best_portfolio.sharpe_ratio}\n")
        f.write(f"  Sortino ratio: {best_portfolio.sortino_ratio}\n")
        f.write(f"  Calmar ratio: {best_portfolio.calmar_ratio}\n")
    
   
    fig4 = optuna.visualization.plot_contour(study, params=['l1_coef', 'l2_coef'])
    fig5 = optuna.visualization.plot_contour(study, params=['holding_period', 'fitting_period'])
    
    # Save the best model parameters
    import joblib
    joblib.dump(study.best_params, "best_portfolio_params.pkl")

[I 2025-04-22 14:57:19,046] A new study created in memory with name: no-name-33bdc13b-1852-44d6-b2e3-7b263ec67224
[I 2025-04-22 14:57:20,540] Trial 0 finished with value: 0.2910101207446953 and parameters: {'l1_coef': 0.002429977693554593, 'l2_coef': 0.0006720091466018288, 'holding_period': 13, 'fitting_period': 203}. Best is trial 0 with value: 0.2910101207446953.
c:\Users\bodya\anaconda3\Lib\site-packages\skfolio\moments\covariance\_base.py:102: UserWarning:

The covariance matrix is not positive definite. The Clipping algorithm will be used to find the nearest positive definite covariance.

c:\Users\bodya\anaconda3\Lib\site-packages\skfolio\moments\covariance\_base.py:102: UserWarning:

The covariance matrix is not positive definite. The Clipping algorithm will be used to find the nearest positive definite covariance.

c:\Users\bodya\anaconda3\Lib\site-packages\skfolio\moments\covariance\_base.py:102: UserWarning:

The covariance matrix is not positive definite. The Clipping algorit

Best parameters: {'l1_coef': 1.132971953693404e-05, 'l2_coef': 1.4258748847106347e-05, 'holding_period': 6, 'fitting_period': 95}
Best average drawdown: 0.17341711077835087
Mean return: 0.00027077962057540244
Max drawdown: 0.4590395094594891
Average drawdown: 0.17341711077835087
Sharpe ratio: 0.025618626473662275
Sortino ratio: 0.03628391041866885
Calmar ratio: 0.0005898830383777654


In [67]:
show(fig4)

In [68]:
show(fig5)